In [1]:
import sys
sys.path.append("/home/ali/Desktop/python/Machine Learning/Dataset_custom")

In [2]:
from load_data import LoadModel
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

ModuleNotFoundError: No module named 'load_data'

In [306]:
train, test = LoadModel().load_hpart()
train_y = train["SalePrice"]
train = train.drop('SalePrice', axis=1)

In [307]:
def fillNAn(data):
    nancolumns = data.columns[data.isna().any()].tolist() # data which has at least one missing data
    for i in nancolumns:
        if data[i].dtypes == 'float64':
            data[i].fillna(data[i].mean(), inplace=True) #fill numerical data with mean of others
        else:
            data[i].fillna(data[i].mode()[0], inplace=True) #fill non-numerical data with most repetitive data
    return data

In [308]:
def encode(data):
    enc = LabelEncoder()
    for i in data.columns:
        if data[i].dtypes == 'O':
           data[i] = enc.fit_transform(data[i])
    return data

In [309]:
fillNantransform = FunctionTransformer(fillNAn)
encodtransform = FunctionTransformer(encode)

In [310]:
pip = Pipeline([('fill Missing data', fillNantransform), ('lable encoding', encodtransform), ('Normalize', StandardScaler()),('lassomodel', Lasso(fit_intercept=True, tol=1e-3))])
 # ('model', LinearRegression()), 

In [311]:
# # show correlation between different variables
# corr = train.corr()
# f, ax = plt.subplots(figsize=(300, 250))
# mask = np.triu(np.ones_like(corr, dtype=bool))
# cmap = sns.diverging_palette(230, 20, as_cmap=True)
# sns.heatmap(corr, annot=True, mask = mask, cmap=cmap)

In [312]:
# train, train_y = StandardScaler().fit_transform(train), StandardScaler().fit_transform(train_y.values.reshape(-1,1))

In [313]:
# model = LinearRegression()

In [314]:
pip.fit(train, train_y)

Pipeline(steps=[('fill Missing data',
                 FunctionTransformer(func=<function fillNAn at 0x7f77b2b2cca0>)),
                ('lable encoding',
                 FunctionTransformer(func=<function encode at 0x7f77b2b2c9d0>)),
                ('Normalize', StandardScaler()),
                ('lassomodel', Lasso(tol=0.001))])

In [315]:
prediction = pip.predict(test)

In [316]:
output = pd.DataFrame({'Id': test.Id, 'SalePrice': prediction})

In [317]:
output.to_csv('submission.csv', index=False)